<a href="https://colab.research.google.com/github/heltonleiras/Downloader-torrent-para-o-Google-Drive-heltonleiras/blob/main/Downloader%20torrent%20para%20o%20Google%20Drive%20%7C%7C%40heltonleiras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloader de torrent para o Google Drive

**Important Observação:** **Para obter mais espaço em disco:**
Você terá cerca de 384 GB para baixar qualquer torrent que desejar.

### Instale o libtorrent e inicialize a sessão

In [7]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-libtorrent is already the newest version (1.1.5-1build1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


### Mount Google Drive
To stream files we need to mount Google Drive.

In [8]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Adicionar do arquivo torrent
Você pode executar esta célula para adicionar mais arquivos quantas vezes quiser

In [9]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

IndexError: ignored

### Adicionar do link magnético
Você pode executar esta célula para adicionar mais arquivos quantas vezes quiser

In [11]:
params = {"save_path": "/content/drive/My Drive/Torrent"}

while True:
    magnet_link = input("Insira o link magnético ou digite sair: ")
    if magnet_link.lower() == "sair":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


Insira o link magnético ou digite Sair: magnet:?xt=urn:btih:c6a331ba3407a88f6b7c36f0b42fe0c3c070d4e2&dn=VACATORRENT.COM.-DUBLADO-DUAL-AUDIO-.Ms.Marvel.S01E01.1080p.WEB.DUAL&tr=udp%3a%2f%2ftracker.openbittorrent.com%3a80%2fannounce&tr=udp%3a%2f%2ftracker.publicbt.com%3a80%2fannounce&tr=udp%3a%2f%2ftracker.internetwarriors.net%3a1337%2fannounce&tr=udp%3a%2f%2ftracker.opentrackr.org%3a1337%2fannounce&tr=udp%3a%2f%2fexodus.desync.com%3a6969%2fannounce&tr=udp%3a%2f%2fopen.stealth.si%3a80%2fannounce&tr=udp%3a%2f%2fwww.torrent.eu.org%3a451%2fannounce
Insira o link magnético ou digite Sair: sair


### Iniciar download
Aproveite!!! @heltonleiras

In [12]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "na fila",
    "verificando",
    "baixando metadados",
    "baixando",
    "finalizado",
    "semeando",
    "alocando",
    "verificando currículo rápido",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)


FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

Ms.Marvel.S01E01.1080p.WEB.DUAL complete
